# Zero-Cost Proxy Evaluation  w/ minibatch

Testing zero-cost proxies with zone-agnostic behavior policy model

In [1]:
import numpy as np
import sys
import matplotlib.pyplot as plt
import pandas as pd
import pickle
sys.path.insert(0, "../")
import torch
from ppo import PPO
import glob
from tqdm.notebook import tqdm
import json
from scipy.stats import spearmanr

In [2]:
log_data = pd.read_csv("../data/rule_based_log_data/15zone/0_cleaned_log.csv")
with open("../data/rule_based_log_data/15zone/action_probs_all_data.pkl", "rb") as f:
    behavior_model = pickle.load(f)
# with open("../data/invalid_policy_list_20220705.json") as f:
#     invalid_policies = json.load(f)["invalid_policies"]
invalid_policies = []

## 30 Day MiniBatch

In [3]:
num_ts_per_day = 4 * 24
num_days = 15
ts_end = num_ts_per_day * num_days
zones = log_data["zone"].unique()

## Loading all Policies

In [4]:
policy_list = sorted(list(glob.glob(f"../policy_library_20220820/**.pth")))

### 1. GradNorm

In [5]:
from zero_cost_proxies.grad_norm import GradNorm
policy_scores = {}
for zone in zones:
    print(zone)
    ope_data = log_data[log_data["zone"] == zone].sort_values(by=["timestep"])
    ope_data = log_data[:ts_end]
    for policy in tqdm(policy_list):
        agent = PPO(6, 1, 0.003, 0.0005, 1, 10, 0.2,
                    has_continuous_action_space=True, action_std_init=0.2, 
                    device=torch.device('cpu'), diverse_policies=list(),
                    diverse_weight=0, diverse_increase=True)
        agent.load(policy)
        agent.policy_evaluation = True
        agent.policy_old.set_action_std(0.1)
        gn = GradNorm(agent, behavior_model)
        if policy not in invalid_policies:
            norm = gn.get_grad_norm(ope_data)
        else:
            continue
        if policy not in policy_scores:
            policy_scores[policy] = {}
        if zone not in policy_scores[policy]:
            policy_scores[policy][zone] = norm

Perimeter_top_ZN_3


  0%|          | 0/870 [00:00<?, ?it/s]

/Users/aakashsasikumar/Documents/MSc/Research/Dissimilar Policy/building-MARL/studies/../ope/iw.py:61: RuntimeWarning: divide by zero encountered in log
  return np.log(value / (1- value))


Perimeter_bot_ZN_1


  0%|          | 0/870 [00:00<?, ?it/s]

...

  0%|          | 0/870 [00:00<?, ?it/s]

Perimeter_mid_ZN_1


  0%|          | 0/870 [00:00<?, ?it/s]

Perimeter_bot_ZN_4


  0%|          | 0/870 [00:00<?, ?it/s]

In [6]:
eval_data_loc = "../data/eval_data/evaluation_report_20220820.csv"
df = pd.read_csv(eval_data_loc, header=None, names=["datetime","policy","zone","energy"])
spearman_corr = {}
for zone in zones:
    eval_df = df[df["zone"] == zone]

    # invalid_policies = list(set(policy_list) - set(policy_scores.keys()))
    for i_policy in invalid_policies:
        eval_df = eval_df[eval_df["policy"]!=i_policy[3:]]
    eval_df = eval_df.sort_values(by=["energy"])
    score_list = []
    for i, row in eval_df.iterrows():
        score_list.append(policy_scores[f"../{row['policy']}"][zone])

    eval_df["ope_scores"] = score_list
    correlation = spearmanr(eval_df["energy"].values, eval_df["ope_scores"].values)
    spearman_corr[zone] = correlation

In [7]:
spearman_corr

{'Perimeter_top_ZN_3': SpearmanrResult(correlation=-0.6865461379890686, pvalue=2.830810891363366e-122),
 'Perimeter_bot_ZN_1': SpearmanrResult(correlation=-0.6369612189430071, pvalue=3.4672597559123844e-100),
 'Perimeter_top_ZN_1': SpearmanrResult(correlation=-0.6257318273268654, pvalue=9.817431228826295e-96),
 'Core_bottom': SpearmanrResult(correlation=-0.6209011775719514, pvalue=7.106159124851904e-94),
 'Perimeter_top_ZN_4': SpearmanrResult(correlation=-0.6330619441564675, pvalue=1.2787197136151444e-98),
 'Core_top': SpearmanrResult(correlation=-0.6300092341279773, pvalue=2.0786685537682936e-97),
 'Perimeter_bot_ZN_2': SpearmanrResult(correlation=-0.6305946149171981, pvalue=1.2207197934204085e-97),
 'Perimeter_bot_ZN_3': SpearmanrResult(correlation=-0.7265522434160954, pvalue=1.2891916550609763e-143),
 'Perimeter_mid_ZN_4': SpearmanrResult(correlation=-0.6322384578781809, pvalue=2.7213989812298362e-98),
 'Core_mid': SpearmanrResult(correlation=-0.6222749693766, pvalue=2.1187868081180

In [8]:
with open(f"data/15zone/grad_norm/raw_scores/grad_norm_raw_scores_{num_days}_days_18_09_2022.pkl", "wb+") as f:
    pickle.dump(policy_scores, f)

In [9]:
with open(f"data/15zone/grad_norm/spearman_corr/grad_norm_spearman_corrs_{num_days}_days_18_09_2022_new_policies.pkl", "wb+") as f:
    pickle.dump(spearman_corr, f)

### 2. Synflow

In [10]:
from zero_cost_proxies.synflow import SynFlow
policy_scores_synflow = {}
for zone in zones:
    print(zone)
    ope_data = log_data[log_data["zone"] == zone].sort_values(by=["timestep"])
    ope_data = log_data[:ts_end]
    for policy in tqdm(policy_list):
        agent = PPO(6, 1, 0.003, 0.0005, 1, 10, 0.2,
                    has_continuous_action_space=True, action_std_init=0.2, 
                    device=torch.device('cpu'), diverse_policies=list(),
                    diverse_weight=0, diverse_increase=True)
        agent.load(policy)
        agent.policy_evaluation = True
        agent.policy_old.set_action_std(0.1)
        sf = SynFlow(agent)
        if policy not in invalid_policies:
            synflow = sf.get_synflow()
        else:
            continue
        if policy not in policy_scores_synflow:
            policy_scores_synflow[policy] = {}
        if zone not in policy_scores_synflow[policy]:
            policy_scores_synflow[policy][zone] = synflow

Perimeter_top_ZN_3


  0%|          | 0/400 [00:00<?, ?it/s]

Perimeter_bot_ZN_1


  0%|          | 0/400 [00:00<?, ?it/s]

Perimeter_top_ZN_1


...

  0%|          | 0/400 [00:00<?, ?it/s]

Perimeter_mid_ZN_1


  0%|          | 0/400 [00:00<?, ?it/s]

Perimeter_bot_ZN_4


  0%|          | 0/400 [00:00<?, ?it/s]

In [11]:
eval_data_loc = "../data/evaluation_clean_20220705.csv"
df = pd.read_csv(eval_data_loc, header=None, names=["datetime","policy","zone","energy"])
spearman_corr_synflow = {}
for zone in zones:
    eval_df = df[df["zone"] == zone]

    # invalid_policies = list(set(policy_list) - set(policy_scores.keys()))
    for i_policy in invalid_policies:
        eval_df = eval_df[eval_df["policy"]!=i_policy[3:]]
    eval_df = eval_df.sort_values(by=["energy"])
    score_list = []
    for i, row in eval_df.iterrows():
        score_list.append(policy_scores_synflow[f"../{row['policy']}"][zone])

    eval_df["ope_scores"] = score_list
    correlation = spearmanr(eval_df["energy"].values, eval_df["ope_scores"].values)
    spearman_corr_synflow[zone] = correlation

In [12]:
spearman_corr_synflow

{'Perimeter_top_ZN_3': SpearmanrResult(correlation=0.11754028498214543, pvalue=0.0588871083884381),
 'Perimeter_bot_ZN_1': SpearmanrResult(correlation=0.07900484179553947, pvalue=0.20505007316503912),
 'Perimeter_top_ZN_1': SpearmanrResult(correlation=0.023572844503077058, pvalue=0.7057377975989334),
 'Core_bottom': SpearmanrResult(correlation=0.049360067964719125, pvalue=0.4289352248218832),
 'Perimeter_top_ZN_4': SpearmanrResult(correlation=0.043202492039701335, pvalue=0.4887875465978697),
 'Core_top': SpearmanrResult(correlation=0.00984866798820287, pvalue=0.8746644579048296),
 'Perimeter_bot_ZN_2': SpearmanrResult(correlation=0.0915430892175078, pvalue=0.14177663002683502),
 'Perimeter_bot_ZN_3': SpearmanrResult(correlation=0.14188809305088373, pvalue=0.02237182255749148),
 'Perimeter_mid_ZN_4': SpearmanrResult(correlation=0.05661723568700312, pvalue=0.3641482615793461),
 'Core_mid': SpearmanrResult(correlation=-0.0016093271907225396, pvalue=0.9794372731667248),
 'Perimeter_mid_ZN_

In [13]:
with open("data/synflow_raw_scores_15_days_19_07_2022.pkl", "wb+") as f:
    pickle.dump(policy_scores_synflow, f)

In [14]:
with open("data/synflow_spearman_corr_15_days_19_07_2022_new_policies.pkl", "wb+") as f:
    pickle.dump(spearman_corr_synflow, f)

### 3. Jacobian Covariance

In [15]:
from zero_cost_proxies.jacob_cov import JacobianCovariance
policy_scores_jc = {}
for zone in zones:
    print(zone)
    ope_data = log_data[log_data["zone"] == zone].sort_values(by=["timestep"])
    ope_data = log_data[:ts_end]
    states = []
    for i, row in ope_data.iterrows():
        state_vars = ["outdoor_temp", "solar_irradiation", "time_hour",
                      "zone_humidity", "zone_temp", "zone_occupancy"]
        state = [row[var] for var in state_vars]
        states.append(state)
    states = torch.Tensor(states)
    for policy in tqdm(policy_list):
        agent = PPO(6, 1, 0.003, 0.0005, 1, 10, 0.2,
                    has_continuous_action_space=True, action_std_init=0.2, 
                    device=torch.device('cpu'), diverse_policies=list(),
                    diverse_weight=0, diverse_increase=True)
        agent.load(policy)
        agent.policy_evaluation = True
        agent.policy_old.set_action_std(0.1)
        jc = JacobianCovariance(agent)
        if policy not in invalid_policies:
            synflow = jc.compute_jacob_cov(states)
        else:
            continue
        if policy not in policy_scores_jc:
            policy_scores_jc[policy] = {}
        if zone not in policy_scores_jc[policy]:
            policy_scores_jc[policy][zone] = synflow

Perimeter_top_ZN_3


  0%|          | 0/400 [00:00<?, ?it/s]

Perimeter_bot_ZN_1


  0%|          | 0/400 [00:00<?, ?it/s]

Perimeter_top_ZN_1


...

  0%|          | 0/400 [00:00<?, ?it/s]

Perimeter_mid_ZN_1


  0%|          | 0/400 [00:00<?, ?it/s]

Perimeter_bot_ZN_4


  0%|          | 0/400 [00:00<?, ?it/s]

In [16]:
eval_data_loc = "../data/evaluation_clean_20220705.csv"
df = pd.read_csv(eval_data_loc, header=None, names=["datetime","policy","zone","energy"])
spearman_corr_jc = {}
some_policy = list(policy_scores_jc.keys())[0]
for zone in policy_scores_jc[some_policy].keys():
    if zone == "Core_mid":
        continue
    eval_df = df[df["zone"] == zone]

    # invalid_policies = list(set(policy_list) - set(policy_scores.keys()))
    for i_policy in invalid_policies:
        eval_df = eval_df[eval_df["policy"]!=i_policy[3:]]
    eval_df = eval_df.sort_values(by=["energy"])
    score_list = []
    for i, row in eval_df.iterrows():
        score_list.append(policy_scores_jc[f"../{row['policy']}"][zone])

    eval_df["ope_scores"] = score_list
    correlation = spearmanr(eval_df["energy"].values, eval_df["ope_scores"].values)
    spearman_corr_jc[zone] = correlation

In [17]:
spearman_corr_jc

{'Perimeter_top_ZN_3': SpearmanrResult(correlation=-0.02933326886815259, pvalue=0.638432184704356),
 'Perimeter_bot_ZN_1': SpearmanrResult(correlation=-0.044474067729881675, pvalue=0.47607286509132574),
 'Perimeter_top_ZN_1': SpearmanrResult(correlation=0.010725854911901422, pvalue=0.8636055411255131),
 'Core_bottom': SpearmanrResult(correlation=-0.02849407035453547, pvalue=0.6480712622749698),
 'Perimeter_top_ZN_4': SpearmanrResult(correlation=-0.04309474309474309, pvalue=0.48987324271404387),
 'Core_top': SpearmanrResult(correlation=-0.03652274815065513, pvalue=0.5584601852638471),
 'Perimeter_bot_ZN_2': SpearmanrResult(correlation=-0.019529496273682317, pvalue=0.7544272388978923),
 'Perimeter_bot_ZN_3': SpearmanrResult(correlation=0.0062128317942271435, pvalue=0.9207385949318699),
 'Perimeter_mid_ZN_4': SpearmanrResult(correlation=-0.04526560805630573, pvalue=0.4682500739722806),
 'Perimeter_mid_ZN_2': SpearmanrResult(correlation=-0.031678189817724696, pvalue=0.6118225264333249),
 '

In [18]:
with open("data/jacob_cov_spearman_corr_15_days_19_07_2022_new_policies.pkl", "wb+") as f:
    pickle.dump(spearman_corr_jc, f)

In [19]:
with open("data/jacob_cov_raw_scores_15_days_19_07_2022_new_policies.pkl", "wb+") as f:
    pickle.dump(policy_scores_jc, f)